In [1]:
import pandas as pd
import os
import json

In [2]:
data_dir = 'raw_api_output'
files = os.listdir(data_dir)
output_file = 'airports_GeoId.csv'
fout = open(output_file, 'w')
fout.write('iata_code,GeoId\n')

16

In [3]:
for file in files:
    with open('%s/%s'%(data_dir,file)) as f:
        fout.write(file.split('.')[0].split('_')[1] + ',')
        data = json.load(f)
        data = data['result']['geographies']
        geoid = data['Census Blocks'][0]['GEOID'][:12]
        fout.write(geoid + '\n')
fout.close()

In [4]:
import subprocess

In [5]:
proc = subprocess.Popen(['grep', '130510107013','../src_data/cbg2cbg.csv'],stdout=subprocess.PIPE)
tmp = proc.stdout.read()
tmp

b''

In [6]:
len(tmp)

0

In [7]:
fout = open('exist_airport_GeoId.csv', 'w')
ishead = True

In [8]:
with open('airports_GeoId.csv', 'r') as f:
    for line in f.readlines():
        code, id = line.strip().split(',')
        proc = subprocess.Popen(['grep', id,'../src_data/cbg2cbg.csv'],stdout=subprocess.PIPE)
        tmp = proc.stdout.read()
        if (len(tmp) > 0) or ishead:
            fout.write(line)
        ishead = False
fout.close()

In [9]:
us_airports = pd.read_csv('../src_data/large_airports.csv')
us_airports = us_airports[us_airports['iso_country'] == 'US'].reset_index(drop=True)

In [10]:
airport_ids = pd.read_csv('exist_airport_GeoId.csv', dtype=str)
airport_ids

,iata_code,GeoId
0,ANC,020200023011
1,PHX,040131138021
2,CMA,061110056001
3,LAX,060379800281
4,OAK,060014090004
...,...,...
67,IAD,511079801001
68,ORF,517109803001
69,RIC,510879801001
70,SEA,530330284024


In [11]:
mergerd = airport_ids.join(us_airports.set_index("iata_code"), on='iata_code')

In [12]:
mergerd.to_csv('../src_data/airports_geoinfo.csv', index=False)